In [1]:
%pip install pandas requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly
import cufflinks
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import requests
from bs4 import BeautifulSoup

In [3]:
# URL of the CEYPETCO historical prices page
url = 'https://ceypetco.gov.lk/historical-prices/'

# Send a GET request to the URL
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the section containing the historical prices
# This assumes that the data is within a table; adjust the selector as needed
table = soup.find('table')

# Extract table headers
headers = [header.text.strip() for header in table.find_all('th')]

# Extract table rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    columns = row.find_all('td')
    row_data = [col.text.strip() for col in columns]
    rows.append(row_data)

# Create a DataFrame for better data manipulation
df_extracted = pd.DataFrame(rows, columns=headers)

# Display the DataFrame
print(df_extracted)


                  LP 95   LP 92     LAD     LSD      LK     LIK FUR. 800  \
0    01.02.2025  371.00  309.00  286.00  331.00  183.00  191.00   207.00   
1    01.01.2025  371.00  309.00  286.00  313.00  183.00  191.00   207.00   
2    05.12.2024  371.00  309.00  286.00  313.00  188.00  196.00   207.00   
3    01.12.2024  371.00  309.00  286.00  313.00  188.00  191.00   207.00   
4    01.11.2024  371.00  311.00  283.00  313.00  183.00  191.00   207.00   
..          ...     ...     ...     ...     ...     ...     ...      ...   
155  01.01.1991   32.50   30.00   11.00   13.00    8.80    9.85     6.25   
156  26.12.1990   32.50   30.00   11.00   13.00    8.80    9.85     6.30   
157  05.11.1990   38.00   35.00   13.00   15.00    8.80    9.58     6.30   
158  14.08.1990   27.00   25.00   11.00   12.50    8.00    9.85     5.40   
159  01.03.1990   22.00   20.00    9.60   10.90    6.58    9.85     4.70   

    FUR 1500 (High) FUR. 1500 (Low)  
0            207.00          207.00  
1          

In [4]:
df_extracted.rename(columns={df_extracted.columns[0]: 'Date'}, inplace=True)
df_extracted.head()

df_extracted.to_csv('extracted_data.csv', index=False)

In [5]:
df_extracted.dtypes

Date               object
LP 95              object
LP 92              object
LAD                object
LSD                object
LK                 object
LIK                object
FUR. 800           object
FUR 1500 (High)    object
FUR. 1500 (Low)    object
dtype: object

In [7]:
## Type Conversion - Date Format conversion
df_extracted['Date'] = pd.to_datetime(df_extracted['Date'], format='%d.%m.%Y',errors='coerce')
df_extracted = df_extracted.sort_values(by='Date')


In [22]:
df_extracted['LP 95'] = df_extracted['LP 95'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['LP 92'] = df_extracted['LP 92'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['LAD'] = df_extracted['LAD'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['LSD'] = df_extracted['LSD'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['LK'] = df_extracted['LK'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['LIK'] = df_extracted['LIK'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['FUR. 800'] = df_extracted['FUR. 800'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['FUR 1500 (High)'] = df_extracted['FUR 1500 (High)'].apply(pd.to_numeric, errors='coerce').astype(float)
df_extracted['FUR. 1500 (Low)'] = df_extracted['FUR. 1500 (Low)'].apply(pd.to_numeric, errors='coerce').astype(float)


In [13]:
## Type Conversion - Num Format conversion
#df_extracted.iloc[:, 1:10] = df_extracted.iloc[:, 1:10].apply(pd.to_numeric, errors='coerce').astype(float)


In [23]:
print(df_extracted.dtypes)

Date               datetime64[ns]
LP 95                     float64
LP 92                     float64
LAD                       float64
LSD                       float64
LK                        float64
LIK                       float64
FUR. 800                  float64
FUR 1500 (High)           float64
FUR. 1500 (Low)           float64
dtype: object


In [24]:
#Check for NaN values
print(df_extracted.isna().sum())

Date                1
LP 95               0
LP 92               1
LAD                 0
LSD                 0
LK                  0
LIK                 0
FUR. 800            1
FUR 1500 (High)    88
FUR. 1500 (Low)     0
dtype: int64


In [30]:
## Drop the rows with NA and Drop column 'FUR 1500 (High)'

df_preprocessed = df_extracted.drop('FUR 1500 (High)', axis=1)

## Drop rows with missing Date
df_preprocessed =df_preprocessed.dropna(subset ='Date' )

## Fill the NA values of oter colums sing ffill method - previous value
df_preprocessed = df_preprocessed.fillna(method='ffill')


C:\Users\USER\AppData\Local\Temp\ipykernel_3680\1375856138.py:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [33]:
## Check for the dtypes and NA counts of pre_processed data
df_preprocessed.dtypes
df_preprocessed.isna().sum()

Date               datetime64[ns]
LP 95                     float64
LP 92                     float64
LAD                       float64
LSD                       float64
LK                        float64
LIK                       float64
FUR. 800                  float64
FUR. 1500 (Low)           float64
dtype: object

In [37]:
df_preprocessed.info()


<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 159 to 0
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             159 non-null    datetime64[ns]
 1   LP 95            159 non-null    float64       
 2   LP 92            159 non-null    float64       
 3   LAD              159 non-null    float64       
 4   LSD              159 non-null    float64       
 5   LK               159 non-null    float64       
 6   LIK              159 non-null    float64       
 7   FUR. 800         159 non-null    float64       
 8   FUR. 1500 (Low)  159 non-null    float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 12.4 KB


In [36]:
## Check summary stats
df_preprocessed.describe()

,Date,LP 95,LP 92,LAD,LSD,LK,LIK,FUR. 800,FUR. 1500 (Low)
count,159,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000
mean,2011-01-12 13:53:12.452830208,187.125786,161.965409,136.287736,161.027673,91.431950,119.129748,102.075472,102.030126
min,1990-03-01 00:00:00,22.000000,20.000000,9.600000,10.900000,6.580000,9.580000,4.700000,4.290000
25%,2002-09-30 12:00:00,54.500000,50.500000,30.000000,35.300000,22.500000,24.800000,22.450000,21.500000
50%,2009-12-29 00:00:00,133.000000,120.000000,80.000000,95.300000,61.000000,69.000000,53.900000,51.700000
75%,2022-06-09 12:00:00,371.000000,309.000000,284.500000,313.000000,103.500000,191.000000,191.000000,191.000000
max,2025-02-01 00:00:00,550.000000,470.000000,460.000000,520.000000,365.000000,464.000000,419.000000,419.000000
std,NaN,159.874248,130.529085,136.587541,161.877468,94.998857,126.101572,108.355334,108.792543


In [39]:
df_preprocessed.to_csv('output_file.csv', index=False)